In [2]:
%load_ext autoreload
import ast

from event_loop.activity_boundaries import ActivityBoundariesClassifier
from event_loop.activity_type import ActivityTypeClassifier
from event_loop.event_activity import EventActivityAssignment
from event_loop.preprocessing.dataframe import *
from sklearn.metrics import classification_report
import pandas as pd




In [3]:
df_train_in = pd.read_csv('../data/VALID/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})
# This is the Interleaved Data Set for our pipeline
df_il_in = pd.read_csv('../data/PTP-INTERLEAVED/R1/R1.csv', converters={"MessageAttributes": ast.literal_eval})

# Load start and end events from ground truth data.
# Tag according frames in interleaved data for testing
df_gt = pd.read_csv("../data_v3/ptp_ground_truth.csv")

start_indices = df_gt["start"].tolist()
end_indices = df_gt["actual_end"].tolist()

df_train = pre_process(df_train_in)
df_test = pre_process(df_il_in)

df_train = assign_sequence_number(df_train)

df_test["ActivityAction"] = df_test["frame.number"].apply(lambda x: "Activity Start" if x in start_indices else
("Activity End" if x in end_indices else "NoAction"))

df_train = mark_start_end(df_train)

In [ ]:
%autoreload 2
activity_boundaries_classifier = ActivityBoundariesClassifier(df_train, None)

activity_type_classifier = ActivityTypeClassifier(df_train)

event_activity_model = EventActivityAssignment(df_train,10,["sale_order_id", "sale_order_line_id","purchase_requisition_id","purchase_requisition_line_id",])

# Event Loop

In [8]:
records = df_il_in.to_dict("records")

In [29]:
from event_loop.event_activity import *
%autoreload 2

from event_loop.preprocessing.event import keep_event
import time
from event_loop.stack import *
from event_loop.event import Event


EVENT_LOOP_CUTOFF_NO_ACTION = 3
EVENT_LOOP_CUTOFF_END_EVENT = 3
ENTROPY_THRESHOLD = 0.4 #0.5
MAX_WINDOW_SIZE = 10
VERBOSE = False
SETTING = "PTP"

# init variables
event_buffer: list[Event] = []
attribute_buffer: list[dict] = []
stacks: list[Stack] = []
stacks_out: list[Stack] = []
event_loop_index = 0




#activity_model_data = get_activity_model_data(MAX_WINDOW_SIZE)


processing_times = []
buffer_sizes = []

for i, event_data in enumerate(records):
    buffer_sizes.append(sum([len(stack) for stack in stacks]))
    start_time = time.time()
    
    # Filter Event Stream
    if not keep_event(event_data):
        # skip event in loop
        continue
        
    # count every not filtered event for event loop index
    event_loop_index += 1
    
    # Extract Features and generate Event Object
    event = Event(event_data, event_loop_index, event_buffer, SETTING)
    event_buffer.append(event)
    
    activity_boundaries_classifier.classify_event(event) 
    activity_action = event.activity_action
    
    
        # Activity Matching
    if activity_action == "Activity Start": 
        stacks.append(Stack(SETTING,event))
        
    if activity_action == "NoAction": 
        if len(stacks) == 1: 
            stacks[0].append_event(event)
        elif event.origin_request_frame: 
            idx = search_stack_for_request_frame(event.origin_request_frame, stacks)
            stacks[idx].append_event(event)
        else: 
            # Check attributes of each stack
            
            # we can filter out stacks that already have attributes different to the event
            exclude_indices =  event_activity_model.exclude_stacks_by_attribute(stacks, event)
    
            stack_index:int = event_activity_model.check_stack_attributes(stacks, event, exclude_indices)
                    
            if stack_index == -1:        
                stack_index = event_activity_model.assign_to_sequence(event, stacks, 4, exclude_indices)
            
            # for elements that are not matchable based on 2 sequences we fall back to stream index
            if stack_index == -1: 
                stack_index = search_stream_index(stacks, event, exclude_indices)    
            
            # fallback - no match add to first stack
            if stack_index == -1:
                res = next((i for i in range(len(stacks)) if i not in exclude_indices and stacks[i].confidence),-1)
                stack_index = res
                
            stacks[stack_index].append_event(event)
            
            
    if activity_action == "Activity End":
        stack_index = search_stack_for_request_frame(event.origin_request_frame, stacks)
        stacks[stack_index].append_event(event)
        
        if event.confidence: 
            if len(stacks) > 1: 
                stack = stacks.pop(stack_index)
                label = activity_type_classifier.classify_stack(stack)
                stack.label = label 
                stacks_out.append(stack)
                
            else: 
                event.confidence = False
                    
    # Loop through all currently open stacks
    for idx, stack in enumerate(stacks):
        last_event = stack[-1]
        # check for non-confident "No Action" Classifications. These could be "Activity End" Instead
        if not last_event.confidence and last_event.activity_action == "NoAction":
            # If a stack has not been continued for N event loops 
            if event_loop_index - last_event.event_loop_index > EVENT_LOOP_CUTOFF_NO_ACTION:
                label = activity_type_classifier.classify_stack(stack)
                stack.label = label 
                stacks.pop(idx)
                stacks_out.append(stack)
               
                
    for idx, stack in enumerate(stacks): 
        last_event = stack.events[-1]
        if not last_event.confidence and last_event.activity_action == "Activity End": 
            if event_loop_index - last_event.event_loop_index > EVENT_LOOP_CUTOFF_END_EVENT: 
                label = activity_type_classifier.classify_stack(stack)
                stack.label = label            
                # we are now sure to pop the stack.
                stacks.pop(idx)
                stacks_out.append(stack)  
            
                
    end_time = time.time()
    processing_times.append(end_time - start_time)
    
for stack in stacks: 
    label = activity_type_classifier.classify_stack(stack)
    stack.label = label      
    stacks_out.append(stack)        

CreatePurchaseRequest
CreateCallForTender
CreatePurchaseRequest
CreateCallForTender
CreatePurchaseRequest
CreateRfq
CreateCallForTender
CreateRfq
CreatePurchaseRequest
BidSelection
CreateCallForTender
CreateRfq
CreatePurchaseRequest
CreateRfq
BidSelection
NO MATCH RES 26090
BidSelection
CreatePurchaseRequest
CreateRfq
CreatePurchaseOrder
ReceiveGoods
CreateCallForTender
CreateRfq
CreatePurchaseOrder
CreatePurchaseRequest
NO MATCH RES 44627
CreatePurchaseOrder
SubmitPayment
NO MATCH RES 48564
ReceiveGoods
BidSelection
ReceiveGoods
CreatePurchaseOrder
CreateRfq
SubmitPayment
NO MATCH RES 57502
BidSelection
SubmitPayment
CreateRfq
CreateRfq
CreateCallForTender
BidSelection
CreatePurchaseOrder
BidSelection
ReceiveGoods
BidSelection
CreatePurchaseRequest
CreatePurchaseOrder
CreatePurchaseOrder
SubmitPayment
CreateCallForTender
CreateRfq
CreateRfq
SubmitPayment
BidSelection
SubmitPayment
CreateRfq
BidSelection
ReceiveGoods
BidSelection
ReceiveGoods
CreatePurchaseOrder
SubmitPayment
ReceiveGo

In [30]:
# create Evaluation Data Frame
start = [stack[0].frame_number for stack in stacks_out]
end = [stack[-1].frame_number for stack in stacks_out]
label = [stack.label for stack in stacks_out]
case_id = [stack.case_id["id"] if stack.case_id else -1 for stack in stacks_out]
res_df = pd.DataFrame({"start_pred":start, "end_pred":end,"label": label, "case_id": case_id})

eval_df = df_gt[["start", "actual_end","activity_name","bp_id"]].merge(res_df,how="left", left_on ="start", right_on = "start_pred").fillna(-1)
eval_df["pred_true"] = eval_df["actual_end"] == eval_df["end_pred"]
eval_df["label_true"] = eval_df["activity_name"] == eval_df["label"]
print(f"Accuracy of matching start and end sequences: {eval_df['pred_true'].mean()}")

eval_df["bp_id_norm"]= eval_df["bp_id"] - eval_df["bp_id"].min() +1

eval_df["case_id_true"] = eval_df["bp_id_norm"] == eval_df["case_id"]
print(f"Overall matching accuracy: {0.5 + eval_df['pred_true'].mean()/2}")

Accuracy of matching start and end sequences: 0.47619047619047616
Overall matching accuracy: 0.7380952380952381


In [31]:
eval_df

,start,actual_end,activity_name,bp_id,start_pred,end_pred,label,case_id,pred_true,label_true,bp_id_norm,case_id_true
0,96,1322,CreatePurchaseRequest,399,96,1322,CreatePurchaseRequest,1,True,True,1,True
1,1367,1887,CreateCallForTender,399,1367,1887,CreateCallForTender,1,True,True,1,True
2,1940,2793,CreatePurchaseRequest,400,1940,2793,CreatePurchaseRequest,2,True,True,2,True
3,2818,15871,CreateRfq,399,2818,16368,CreateRfq,1,False,True,1,True
4,5563,5965,CreateCallForTender,400,5563,5965,CreateCallForTender,2,True,True,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...
58,100724,104454,SubmitPayment,406,100724,108703,SubmitPayment,8,False,True,8,True
59,101210,105925,CreatePurchaseOrder,408,101210,101734,BidSelection,-1,False,False,10,False
60,106266,108703,SubmitPayment,407,106266,112188,SubmitPayment,9,False,True,9,True
61,108727,109696,ReceiveGoods,408,108727,109696,ReceiveGoods,-1,True,True,10,False


In [32]:
# We evaluate the Activity Type Classification on a per-event Basis
from event_loop.activity_boundaries import extract_labels

test_labels = extract_labels(df_test["ActivityAction"])
# Create a Dataframe with all events and their frame numbers
df_aa_test = pd.DataFrame(df_test[["frame.number", "ActivityAction"]])
# Set type of every event to no action 
df_aa_test["ActivityAction"] = "NoAction"
# Merge Predicted end events by frame number
df_aa_test.loc[df_aa_test["frame.number"].isin(eval_df["end_pred"]), "ActivityAction"] = "Activity End"
# Merge predicted start events by frame number
df_aa_test.loc[df_aa_test["frame.number"].isin(eval_df["start_pred"]), "ActivityAction"] = "Activity Start"
print(classification_report(test_labels, df_aa_test["ActivityAction"]))

                precision    recall  f1-score   support

  Activity End       0.73      0.73      0.73        63
Activity Start       1.00      1.00      1.00        63
      NoAction       1.00      1.00      1.00      3783

      accuracy                           0.99      3909
     macro avg       0.91      0.91      0.91      3909
  weighted avg       0.99      0.99      0.99      3909


In [33]:
print(classification_report(eval_df["activity_name"], eval_df["label"]))

                       precision    recall  f1-score   support

         BidSelection       0.67      0.80      0.73        10
  CreateCallForTender       1.00      0.70      0.82        10
  CreatePurchaseOrder       0.56      0.62      0.59         8
CreatePurchaseRequest       1.00      0.89      0.94         9
            CreateRfq       0.73      0.80      0.76        10
         ReceiveGoods       1.00      0.88      0.93         8
        SubmitPayment       0.78      0.88      0.82         8

             accuracy                           0.79        63
            macro avg       0.82      0.79      0.80        63
         weighted avg       0.82      0.79      0.80        63


In [34]:
print(classification_report(eval_df[eval_df["case_id"] != -1]["bp_id_norm"], eval_df[eval_df["case_id"] != -1]["case_id"],zero_division=np.nan))

              precision    recall  f1-score   support

           1       1.00      0.83      0.91         6
           2       1.00      1.00      1.00         6
           3       1.00      0.71      0.83         7
           4       1.00      1.00      1.00         4
           5       0.67      0.67      0.67         6
           6       0.71      0.71      0.71         7
           7       0.75      0.75      0.75         4
           8       0.67      0.40      0.50         5
           9       0.75      0.50      0.60         6
          10       0.00      0.00       nan         5
          11       0.00       nan       nan         0
          12       0.00       nan       nan         0

    accuracy                           0.66        56
   macro avg       0.63      0.66      0.77        56
weighted avg       0.76      0.66      0.77        56
